# Predicting Home Sales Prices Based on a Variety of Indicators

In [2]:
#importing necessary libraries

import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as sm
import scipy.stats as stats

from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans

In [3]:
df = pd.read_csv("kc_house_data.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [5]:
# show all the columns in dataset
pd.set_option('display.max_columns', None) 

In [6]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## 1. Feature Engineering

In [7]:
df["sqft_total15"] = df["sqft_living15"] + df["sqft_lot15"]
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,6990
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,9329
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,10782
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,6360
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,9303


In [8]:
df["sqft_living_added_reno"] = df["sqft_living15"] - df["sqft_living"]
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,6990,160
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,9329,-880
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,10782,1950
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,6360,-600
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,9303,120


In [9]:
df["reno_living"] = np.where(df["sqft_living_added_reno"] != 0, "Y", "N")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,reno_living
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,6990,160,Y
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,9329,-880,Y
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,10782,1950,Y
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,6360,-600,Y
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,9303,120,Y


In [10]:
df["sqft_lot_added_reno"] = df["sqft_lot15"] - df["sqft_lot"]

In [11]:
df["increase_lot"] = np.where(df["sqft_lot_added_reno"] >0, "Y", "N")
df["decrease_lot"] = np.where(df["sqft_lot_added_reno"] <0, "Y", "N")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,reno_living,sqft_lot_added_reno,increase_lot,decrease_lot
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,6990,160,Y,0,N,N
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,9329,-880,Y,397,Y,N
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,10782,1950,Y,-1938,N,Y
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,6360,-600,Y,0,N,N
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,9303,120,Y,-577,N,Y


In [12]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,sqft_lot_added_reno
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,2.161300e+04
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652,14755.008143,-93.347245,-2.338512e+03
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631,27438.012039,600.811800,2.891140e+04
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000,1721.000000,-8690.000000,-1.225778e+06
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000,6979.000000,-310.000000,-6.800000e+02
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000,9460.000000,0.000000,0.000000e+00
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000,12232.000000,220.000000,4.210000e+02
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000,874620.000000,2310.000000,3.268790e+05


In [13]:
df["basement"] = np.where(df["sqft_basement"] > 0, "Y", "N")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,reno_living,sqft_lot_added_reno,increase_lot,decrease_lot,basement
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,6990,160,Y,0,N,N,N
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,9329,-880,Y,397,Y,N,Y
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,10782,1950,Y,-1938,N,Y,N
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,6360,-600,Y,0,N,N,Y
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,9303,120,Y,-577,N,Y,N


In [14]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,sqft_lot_added_reno
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,2.161300e+04
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652,14755.008143,-93.347245,-2.338512e+03
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631,27438.012039,600.811800,2.891140e+04
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000,1721.000000,-8690.000000,-1.225778e+06
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000,6979.000000,-310.000000,-6.800000e+02
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000,9460.000000,0.000000,0.000000e+00
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000,12232.000000,220.000000,4.210000e+02
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000,874620.000000,2310.000000,3.268790e+05


In [15]:
def year_reno(built, reno):
    if reno == 0:
        return built
    if reno > 1900:
        return reno
    else:
        return "Unknown"
    
df['yr_last_reno'] = df.apply(lambda x: year_reno(x.yr_built, x.yr_renovated), axis=1)
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,reno_living,sqft_lot_added_reno,increase_lot,decrease_lot,basement,yr_last_reno
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,6990,160,Y,0,N,N,N,1955
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,9329,-880,Y,397,Y,N,Y,1991
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,10782,1950,Y,-1938,N,Y,N,1933
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,6360,-600,Y,0,N,N,Y,1965
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,9303,120,Y,-577,N,Y,N,1987


## 2. Removing Outliers

In [16]:
df2 = df.copy()

In [17]:
# 1. price

# creating new column with z-scores
#df2["zscoreprice"] = np.abs(stats.zscore(df2["price"]))

# identifying outliers with z-scores > 3
#outlierPrice = df2.loc[df2["zscoreprice"]>3].index

#print(outlierPrice)

# removing outliers
#df2.drop(df2.loc[df2["zscoreprice"]>3].index, inplace = True)
#df2.info()

In [18]:
# 2. bedrooms

# creating new column with z-scores
df2["zscorebed"] = np.abs(stats.zscore(df2["bedrooms"]))

# identifying outliers with z-scores > 3
outlierBed = df2.loc[df2["zscorebed"]>3]

print(outlierBed)

# outliers have 7+ bedrooms

# removing outliers
df2.drop(df2.loc[df2["zscorebed"]>3].index, inplace = True)
df2.info()

               id             date      price  bedrooms  bathrooms  \
556    5486800070  20140620T000000  1950000.0         7       3.50   
875    6306400140  20140612T000000  1095000.0         0       0.00   
1135   4024100951  20150105T000000   420000.0         7       3.00   
1241   7227802030  20140623T000000   350000.0         7       3.00   
1660   9126101740  20141204T000000   490000.0         8       5.00   
...           ...              ...        ...       ...        ...   
18975  1778360150  20140620T000000  1240000.0         7       5.50   
19254  8812401450  20141229T000000   660000.0        10       3.00   
19302  3756900027  20141125T000000   575000.0         8       3.00   
19327  2771604190  20140617T000000   824000.0         7       4.25   
19452  3980300371  20140926T000000   142000.0         0       0.00   

       sqft_living  sqft_lot  floors  waterfront  view  condition  grade  \
556           4640     15235     2.0           0     1          3     11   
875    

In [19]:
# 6. sqft_total15

# creating new column with z-scores
df2["zscoresqft_total15"] = np.abs(stats.zscore(df2["sqft_total15"]))

# identifying outliers with z-scores > 3
outliersqft_total15 = df2.loc[df2["zscoresqft_total15"]>3].index

print(outliersqft_total15)

df2.drop(df2.loc[df2["zscoresqft_total15"]>3].index, inplace = True)
df2.describe()

Int64Index([    5,    98,   147,   238,   411,   416,   419,   443,   484,
              519,
            ...
            21090, 21287, 21325, 21344, 21351, 21361, 21431, 21486, 21525,
            21548],
           dtype='int64', length=360)


,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,sqft_lot_added_reno,yr_last_reno,zscorebed,zscoresqft_total15
count,2.117800e+04,2.117800e+04,21178.000000,21178.000000,21178.000000,2.117800e+04,21178.000000,21178.000000,21178.000000,21178.000000,21178.000000,21178.000000,21178.000000,21178.000000,21178.000000,21178.000000,21178.000000,21178.000000,21178.000000,21178.000000,21178.000000,21178.000000,2.117800e+04,21178.000000,21178.000000,21178.000000
mean,4.621115e+09,5.374988e+05,3.360516,2.106774,2063.545424,1.196768e+04,1.492374,0.007555,0.229436,3.410993,7.647795,1773.172868,290.372556,1970.837378,84.249315,98078.496836,47.561378,-122.216818,1977.400510,9805.690481,11783.090991,-86.144915,-2.161990e+03,1973.214279,0.777117,0.273797
std,2.875030e+09,3.650871e+05,0.876101,0.760417,898.924053,2.699178e+04,0.540123,0.086593,0.758704,0.651141,1.165242,811.240363,438.722144,29.435703,401.327106,53.679865,0.137704,0.138350,679.460756,10193.156905,10421.513321,585.020848,2.289236e+04,28.882104,0.532497,0.285199
min,1.000102e+06,7.500000e+04,1.000000,0.000000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.514000,460.000000,651.000000,1721.000000,-8110.000000,-1.148064e+06,1900.000000,0.398737,0.000084
25%,2.143701e+09,3.200000e+05,3.000000,1.500000,1420.000000,5.004250e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.474125,-122.329000,1480.000000,5080.000000,6930.000000,-300.000000,-6.610000e+02,1954.000000,0.398737,0.144901
50%,3.975400e+09,4.500000e+05,3.000000,2.250000,1900.000000,7.543000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1550.000000,0.000000,1974.000000,0.000000,98065.000000,47.572900,-122.234000,1830.000000,7560.000000,9400.000000,0.000000,0.000000e+00,1977.000000,0.676485,0.217754
75%,7.346350e+09,6.400000e+05,4.000000,2.500000,2530.000000,1.039150e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2190.000000,560.000000,1997.000000,0.000000,98118.000000,47.678500,-122.129000,2350.000000,9928.500000,12007.750000,220.000000,4.100000e+02,1999.000000,0.676485,0.312315
max,9.900000e+09,7.700000e+06,6.000000,8.000000,12050.000000,1.164794e+06,3.500000,1.000000,4.000000,5.000000,13.000000,8860.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6110.000000,94960.000000,96810.000000,2310.000000,7.651100e+04,2015.000000,2.826929,2.994358


In [20]:
# 3. bathrooms

# creating new column with z-scores
df2["zscorebath"] = np.abs(stats.zscore(df2["bathrooms"]))

# identifying outliers with z-scores > 3
outlierBath = df2.loc[df2["zscorebath"]>3]

print(outlierBath)

# outliers have >4 bathrooms

# removing outliers
df2.drop(df2.loc[df2["zscorebath"]>3].index, inplace = True)
df2.info()

               id             date      price  bedrooms  bathrooms  \
270    4054500390  20141007T000000  1365000.0         4       4.75   
300    3225069065  20140624T000000  3075000.0         4       5.00   
450    4055700030  20150502T000000  1450000.0         3       4.50   
1055   5416300240  20150202T000000   935000.0         4       4.50   
1100   7237500390  20141110T000000  1570000.0         5       4.50   
...           ...              ...        ...       ...        ...   
21283  2937300050  20150227T000000   988990.0         4       4.75   
21494  2413910120  20140702T000000   915000.0         3       4.50   
21506  2524069097  20140509T000000  2238890.0         5       6.50   
21551  1561750040  20141224T000000  1375000.0         5       4.50   
21576  9253900271  20150107T000000  3567000.0         5       4.50   

       sqft_living  sqft_lot  floors  waterfront  view  condition  grade  \
270           5310     57346     2.0           0     0          4     11   
300    

In [21]:
# 4. sqft_living15

# creating new column with z-scores
df2["zscoresqft_living15"] = np.abs(stats.zscore(df2["sqft_living15"]))

# identifying outliers with z-scores > 3
outlierSqft_living15 = df2.loc[df2["zscoresqft_living15"]>3]

print(outlierSqft_living15)

#outliers have 4,000+ sqft living

# removing outliers
df2.drop(df2.loc[df2["zscoresqft_living15"]>3].index, inplace = True)
df2.describe()

               id             date      price  bedrooms  bathrooms  \
21     2524049179  20140826T000000  2000000.0         3       2.75   
384     713500030  20140728T000000  1350000.0         5       3.50   
473    8651610890  20141014T000000  1150000.0         4       3.25   
679    7852070060  20140731T000000  1145000.0         4       3.50   
722    2225059214  20140808T000000  1578000.0         4       3.25   
...           ...              ...        ...       ...        ...   
21333  2424059174  20150508T000000  1999950.0         4       3.25   
21375  6061500100  20140717T000000  1174660.0         6       3.50   
21520  7237550100  20140825T000000  1408760.0         4       4.00   
21540   715010530  20150113T000000  1881580.0         5       3.50   
21590  7430200100  20140514T000000  1222500.0         4       3.50   

       sqft_living  sqft_lot  floors  waterfront  view  condition  grade  \
21            3050     44867     1.0           0     4          3      9   
384    

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,sqft_lot_added_reno,yr_last_reno,zscorebed,zscoresqft_total15,zscorebath,zscoresqft_living15
count,2.080600e+04,2.080600e+04,20806.000000,20806.000000,20806.000000,2.080600e+04,20806.000000,20806.000000,20806.000000,20806.000000,20806.000000,20806.000000,20806.000000,20806.000000,20806.000000,20806.000000,20806.000000,20806.000000,20806.000000,20806.000000,20806.000000,20806.000000,2.080600e+04,20806.000000,20806.000000,20806.000000,20806.000000,20806.000000
mean,4.627585e+09,5.190384e+05,3.340383,2.072731,2017.234644,1.174245e+04,1.484812,0.006729,0.213929,3.414496,7.598049,1734.433577,282.801067,1970.483947,83.451937,98078.971931,47.560872,-122.218156,1943.956118,9644.911708,11588.867827,-73.278525,-2.097540e+03,1972.852927,0.768424,0.271926,0.773290,0.756865
std,2.875668e+09,3.133196e+05,0.864127,0.712069,820.034447,2.676049e+04,0.539353,0.081755,0.730389,0.652925,1.101167,750.562208,424.866784,29.406273,399.503086,53.713948,0.138376,0.138222,624.181684,10005.931992,10203.625368,548.541554,2.272951e+04,28.876991,0.523305,0.279213,0.530020,0.549079
min,1.000102e+06,7.500000e+04,1.000000,0.000000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.514000,460.000000,651.000000,1721.000000,-4880.000000,-1.148064e+06,1900.000000,0.398737,0.000084,0.140418,0.002748
25%,2.154638e+09,3.190000e+05,3.000000,1.500000,1410.000000,5.000000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1180.000000,0.000000,1951.000000,0.000000,98033.000000,47.472100,-122.331000,1480.000000,5040.000000,6870.000000,-290.000000,-6.430000e+02,1954.000000,0.398737,0.145990,0.469193,0.332077
50%,3.975400e+09,4.450000e+05,3.000000,2.250000,1880.000000,7.500000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1540.000000,0.000000,1974.000000,0.000000,98070.000000,47.572550,-122.238000,1820.000000,7519.000000,9340.000000,0.000000,0.000000e+00,1977.000000,0.676485,0.218155,0.517132,0.670880
75%,7.351150e+09,6.270000e+05,4.000000,2.500000,2490.000000,1.022975e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2150.000000,550.000000,1996.000000,0.000000,98118.000000,47.679000,-122.132000,2320.000000,9812.000000,11848.750000,220.000000,4.060000e+02,1998.000000,0.676485,0.312078,1.455518,1.045118
max,9.900000e+09,4.489000e+06,6.000000,4.250000,7620.000000,1.164794e+06,3.500000,1.000000,4.000000,5.000000,13.000000,7320.000000,2850.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,3970.000000,94960.000000,96810.000000,2310.000000,7.651100e+04,2015.000000,2.826929,2.994358,2.818557,2.996650


In [22]:
# 5. sqft_lot15

# creating new column with z-scores
df2["zscoresqft_lot15"] = np.abs(stats.zscore(df2["sqft_lot15"]))

# identifying outliers with z-scores > 3
outlierSqft_lot15 = df2.loc[df2["zscoresqft_lot15"]>3]

print(outlierSqft_lot15)

#outliers have massive, massive lot sizes

# removing outliers
df2.drop(df2.loc[df2["zscoresqft_lot15"]>3].index, inplace = True)
df2.describe()

               id             date      price  bedrooms  bathrooms  \
49      822039084  20150311T000000  1350000.0         3       2.50   
74     3444100400  20150316T000000   349000.0         3       1.75   
79     1531000030  20150323T000000   720000.0         4       2.50   
102    7214720075  20141212T000000   699950.0         3       2.25   
132    1243100136  20140612T000000   784000.0         3       3.50   
...           ...              ...        ...       ...        ...   
20754   323079065  20140624T000000   790000.0         4       3.50   
20843  2225069036  20140815T000000   925000.0         4       3.25   
21343  2924079034  20140925T000000   332220.0         3       1.50   
21386   774101755  20150417T000000   320000.0         3       1.75   
21532   324069112  20140617T000000  1325000.0         4       4.00   

       sqft_living  sqft_lot  floors  waterfront  view  condition  grade  \
49            2753     65005     1.0           1     2          5      9   
74     

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,sqft_lot_added_reno,yr_last_reno,zscorebed,zscoresqft_total15,zscorebath,zscoresqft_living15,zscoresqft_lot15
count,2.023400e+04,2.023400e+04,20234.000000,20234.000000,20234.000000,2.023400e+04,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,2.023400e+04,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000
mean,4.681076e+09,5.165755e+05,3.337452,2.066559,2001.704705,9.701201e+03,1.484704,0.006178,0.211921,3.413512,7.584363,1719.304833,282.399872,1970.249234,82.762034,98079.819017,47.561386,-122.222839,1930.780123,8335.162153,10265.942275,-70.924582,-1.366039e+03,1972.616141,0.769929,0.234599,0.773944,0.752945,0.413367
std,2.872584e+09,3.126006e+05,0.865601,0.712002,810.947230,1.842295e+04,0.540918,0.078357,0.726917,0.653364,1.091442,740.017408,422.135226,29.598615,397.941970,54.012594,0.137652,0.134945,619.224293,5837.142971,6078.770637,542.367782,1.678425e+04,29.097122,0.524115,0.144581,0.529656,0.543518,0.431960
min,1.000102e+06,7.800000e+04,1.000000,0.000000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.514000,460.000000,651.000000,1721.000000,-4880.000000,-1.148064e+06,1900.000000,0.398737,0.000084,0.140418,0.002748,0.000191
25%,2.214801e+09,3.153188e+05,3.000000,1.500000,1400.000000,5.000000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1180.000000,0.000000,1951.000000,0.000000,98033.000000,47.474300,-122.332000,1470.000000,5000.000000,6805.250000,-290.000000,-6.000000e+02,1953.000000,0.398737,0.143317,0.469193,0.332077,0.155007
50%,4.031000e+09,4.410000e+05,3.000000,2.250000,1870.000000,7.380000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1520.000000,0.000000,1973.000000,0.000000,98072.000000,47.573000,-122.249000,1800.000000,7450.000000,9250.000000,0.000000,0.000000e+00,1977.000000,0.676485,0.214470,0.517132,0.670880,0.327555
75%,7.436500e+09,6.250000e+05,4.000000,2.500000,2470.000000,9.928000e+03,2.000000,0.000000,0.000000,4.000000,8.000000,2120.000000,550.000000,1996.000000,0.000000,98118.000000,47.678800,-122.144000,2300.000000,9608.000000,11574.750000,220.000000,3.950000e+02,1999.000000,0.676485,0.305026,1.455518,1.045118,0.533163
max,9.900000e+09,4.489000e+06,6.000000,4.250000,7620.000000,1.164794e+06,3.500000,1.000000,4.000000,5.000000,13.000000,7320.000000,2850.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,3970.000000,39639.000000,43174.000000,2310.000000,3.387700e+04,2015.000000,2.826929,1.037243,2.818557,2.996650,2.997703


In [23]:
# 6. yr_built

# creating new column with z-scores
df2["zscoreyr_built"] = np.abs(stats.zscore(df2["yr_built"]))

# identifying outliers with z-scores > 3
outlierYr_built = df2.loc[df2["zscoreyr_built"]>3]

print(outlierYr_built)

# no outliers for year built

Empty DataFrame
Columns: [id, date, price, bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition, grade, sqft_above, sqft_basement, yr_built, yr_renovated, zipcode, lat, long, sqft_living15, sqft_lot15, sqft_total15, sqft_living_added_reno, reno_living, sqft_lot_added_reno, increase_lot, decrease_lot, basement, yr_last_reno, zscorebed, zscoresqft_total15, zscorebath, zscoresqft_living15, zscoresqft_lot15, zscoreyr_built]
Index: []


In [24]:
df2.drop(columns = ["id", 'yr_last_reno', 'zscorebed',
       'zscoresqft_total15', 'zscorebath', 'zscoresqft_living15',
       'zscoresqft_lot15', 'zscoreyr_built'], inplace = True)

In [25]:
df2.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,sqft_lot_added_reno
count,2.023400e+04,20234.000000,20234.000000,20234.000000,2.023400e+04,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,20234.000000,2.023400e+04
mean,5.165755e+05,3.337452,2.066559,2001.704705,9.701201e+03,1.484704,0.006178,0.211921,3.413512,7.584363,1719.304833,282.399872,1970.249234,82.762034,98079.819017,47.561386,-122.222839,1930.780123,8335.162153,10265.942275,-70.924582,-1.366039e+03
std,3.126006e+05,0.865601,0.712002,810.947230,1.842295e+04,0.540918,0.078357,0.726917,0.653364,1.091442,740.017408,422.135226,29.598615,397.941970,54.012594,0.137652,0.134945,619.224293,5837.142971,6078.770637,542.367782,1.678425e+04
min,7.800000e+04,1.000000,0.000000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.514000,460.000000,651.000000,1721.000000,-4880.000000,-1.148064e+06
25%,3.153188e+05,3.000000,1.500000,1400.000000,5.000000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1180.000000,0.000000,1951.000000,0.000000,98033.000000,47.474300,-122.332000,1470.000000,5000.000000,6805.250000,-290.000000,-6.000000e+02
50%,4.410000e+05,3.000000,2.250000,1870.000000,7.380000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1520.000000,0.000000,1973.000000,0.000000,98072.000000,47.573000,-122.249000,1800.000000,7450.000000,9250.000000,0.000000,0.000000e+00
75%,6.250000e+05,4.000000,2.500000,2470.000000,9.928000e+03,2.000000,0.000000,0.000000,4.000000,8.000000,2120.000000,550.000000,1996.000000,0.000000,98118.000000,47.678800,-122.144000,2300.000000,9608.000000,11574.750000,220.000000,3.950000e+02
max,4.489000e+06,6.000000,4.250000,7620.000000,1.164794e+06,3.500000,1.000000,4.000000,5.000000,13.000000,7320.000000,2850.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,3970.000000,39639.000000,43174.000000,2310.000000,3.387700e+04


In [26]:
def year_reno(built, reno):
    if reno == 0:
        return built
    if reno > 1900:
        return reno
    else:
        return "Unknown"
    
df2['yr_last_reno'] = df2.apply(lambda x: year_reno(x.yr_built, x.yr_renovated), axis=1)
df2.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,reno_living,sqft_lot_added_reno,increase_lot,decrease_lot,basement,yr_last_reno
0,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,6990,160,Y,0,N,N,N,1955
1,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,9329,-880,Y,397,Y,N,Y,1991
2,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,10782,1950,Y,-1938,N,Y,N,1933
3,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,6360,-600,Y,0,N,N,Y,1965
4,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,9303,120,Y,-577,N,Y,N,1987


In [27]:
pd.set_option('display.max_rows', None)

In [28]:
df2["yr_last_reno"].value_counts()

2014    629
2005    451
2006    436
2003    425
2007    413
2004    412
1977    386
2008    367
1968    355
1978    348
1967    329
1979    321
1959    317
1990    300
1954    287
2001    284
1962    283
1987    282
1989    264
1969    260
1955    253
1988    246
1994    243
2009    243
1999    242
1947    239
1963    236
1960    236
1966    235
1976    228
1998    225
1985    223
2013    223
1984    222
2002    219
1948    214
1950    214
1986    213
2000    213
1951    212
1958    211
1980    211
1983    208
1961    208
1991    205
1993    202
1952    199
1953    199
1942    198
1992    196
1956    190
1957    188
1996    180
1975    179
2012    178
1981    177
1965    175
1949    173
1997    166
1964    163
1926    161
1995    159
2010    156
1943    150
1925    146
1974    146
1941    143
1940    142
1973    140
2011    140
1972    135
1944    128
1970    126
1924    117
1946    115
1928    115
1910    114
1929    109
1918    106
1927    105
1982    102
1939     95
1971     94
1945

## 3. Correlations & Analysis

In [29]:
df2.corr()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,sqft_lot_added_reno,yr_last_reno
price,1.000000,0.295015,0.467888,0.660500,0.076740,0.251837,0.242742,0.386249,0.057986,0.649302,0.553523,0.298516,0.023485,0.135995,-0.027930,0.349604,-0.002704,0.561846,0.141293,0.192910,-0.346115,-0.035094,0.079617
bedrooms,0.295015,1.000000,0.504723,0.601685,0.060449,0.165878,-0.014706,0.071008,0.034042,0.355370,0.488518,0.299485,0.165057,0.016884,-0.164436,-0.030064,0.144845,0.409988,0.150920,0.186686,-0.431553,-0.013865,0.174832
bathrooms,0.467888,0.504723,1.000000,0.722848,0.046836,0.510460,0.043125,0.154422,-0.121054,0.639754,0.647856,0.252921,0.526049,0.047668,-0.203551,0.010111,0.222845,0.547796,0.078936,0.131601,-0.455380,-0.023956,0.557029
sqft_living,0.660500,0.601685,0.722848,1.000000,0.146201,0.345008,0.082589,0.260913,-0.044986,0.735898,0.855722,0.420951,0.313690,0.058093,-0.189390,0.044749,0.221612,0.743701,0.275444,0.340253,-0.646110,-0.064683,0.340611
sqft_lot,0.076740,0.060449,0.046836,0.146201,1.000000,-0.060345,0.053085,0.042675,0.004023,0.095389,0.143533,0.029243,0.007743,0.010495,-0.105409,-0.050908,0.143390,0.137919,0.426672,0.423761,-0.061137,-0.949248,0.005539
floors,0.251837,0.165878,0.510460,0.345008,-0.060345,1.000000,0.022053,0.017609,-0.265013,0.457370,0.533091,-0.271745,0.494759,0.006421,-0.058372,0.040145,0.127375,0.267836,-0.153468,-0.120084,-0.210066,0.012864,0.510277
waterfront,0.242742,-0.014706,0.043125,0.082589,0.053085,0.022053,1.000000,0.394384,0.009954,0.069901,0.056572,0.059486,-0.025512,0.084441,0.040016,-0.011564,-0.031374,0.081498,0.105419,0.109531,-0.030440,-0.021605,-0.002732
view,0.386249,0.071008,0.154422,0.260913,0.042675,0.017609,0.394384,1.000000,0.048791,0.226458,0.133594,0.267033,-0.069301,0.103277,0.109218,0.021617,-0.102766,0.264532,0.099293,0.122293,-0.088099,-0.012310,-0.033950
condition,0.057986,0.034042,-0.121054,-0.044986,0.004023,-0.265013,0.009954,0.048791,1.000000,-0.141067,-0.156668,0.188224,-0.358916,-0.062294,-0.002494,-0.005248,-0.099383,-0.085918,0.061638,0.050436,-0.030830,0.017020,-0.393372
grade,0.649302,0.355370,0.639754,0.735898,0.095389,0.457370,0.069901,0.226458,-0.141067,1.000000,0.727211,0.138880,0.447123,0.016508,-0.173595,0.111619,0.180512,0.683033,0.198886,0.260559,-0.320490,-0.035535,0.462126


In [30]:
df2["price"].groupby(df2["bedrooms"]).mean()

bedrooms
1    315273.585106
2    399564.344158
3    459458.563965
4    607662.017066
5    703078.434201
6    691149.330275
Name: price, dtype: float64

In [31]:
df2["price"].groupby(df2["bathrooms"]).mean()

bathrooms
0.00    3.820000e+05
0.50    2.373750e+05
0.75    2.870076e+05
1.00    3.472798e+05
1.25    6.212167e+05
1.50    4.092252e+05
1.75    4.548317e+05
2.00    4.554292e+05
2.25    5.296977e+05
2.50    5.451495e+05
2.75    6.582140e+05
3.00    7.001272e+05
3.25    9.326135e+05
3.50    8.911305e+05
3.75    1.171242e+06
4.00    1.220279e+06
4.25    1.490396e+06
Name: price, dtype: float64

In [32]:
df2["price"].groupby(df2["grade"]).mean()

grade
3     2.710000e+05
4     2.113096e+05
5     2.446587e+05
6     3.002513e+05
7     4.011155e+05
8     5.413541e+05
9     7.723272e+05
10    1.047838e+06
11    1.463826e+06
12    2.182000e+06
13    2.432000e+06
Name: price, dtype: float64

In [33]:
df2["bedrooms"].groupby(df2["yr_last_reno"]).mean()

yr_last_reno
1900    3.194030
1901    3.307692
1902    3.560000
1903    3.000000
1904    2.763158
1905    3.285714
1906    3.097561
1907    3.200000
1908    2.971429
1909    3.116883
1910    2.973684
1911    3.568966
1912    3.107692
1913    3.565217
1914    3.162791
1915    3.040000
1916    2.971014
1917    3.078431
1918    2.773585
1919    2.943662
1920    2.942529
1921    3.289855
1922    2.947368
1923    2.876712
1924    2.811966
1925    3.184932
1926    3.155280
1927    3.123810
1928    3.121739
1929    3.311927
1930    3.223881
1931    3.290909
1932    2.793103
1933    3.000000
1934    3.000000
1935    2.823529
1936    2.875000
1937    3.039216
1938    2.928571
1939    2.757895
1940    2.950704
1941    2.755245
1942    2.656566
1943    2.760000
1944    2.960938
1945    2.921348
1946    2.956522
1947    2.794979
1948    2.813084
1949    2.745665
1950    2.943925
1951    2.910377
1952    3.015075
1953    3.115578
1954    3.139373
1955    3.371542
1956    3.294737
1957    3.393617
1

In [34]:
df2["price"].groupby(df2["condition"]).mean()

condition
1    341067.241379
2    326013.070513
3    513826.894270
4    505684.969271
5    596057.554871
Name: price, dtype: float64

In [35]:
df2["yr_last_reno"].groupby(df2["condition"]).mean()

condition
1    1933.275862
2    1948.358974
3    1981.852865
4    1958.639873
5    1947.096794
Name: yr_last_reno, dtype: float64

In [36]:
df2["floors"].groupby(df2["condition"]).mean()

condition
1    1.155172
2    1.147436
3    1.612529
4    1.249485
5    1.265413
Name: floors, dtype: float64

In [37]:
df2["price"].groupby(df2["floors"]).mean()

floors
1.0    433897.195882
1.5    554145.092829
2.0    609243.248666
2.5    937096.275362
3.0    555382.305882
3.5    511916.666667
Name: price, dtype: float64

In [38]:
df2["sqft_living_added_reno"].groupby(df2["grade"]).mean()

grade
3      605.000000
4      829.230769
5      386.426667
6      192.187658
7       -7.424435
8     -109.378411
9     -236.364825
10    -459.536295
11    -857.123894
12   -1689.166667
13   -4140.000000
Name: sqft_living_added_reno, dtype: float64

In [39]:
df2["price"].groupby(df2["zipcode"]).mean()

zipcode
98001    2.765110e+05
98002    2.326047e+05
98003    2.924628e+05
98004    1.234478e+06
98005    7.647649e+05
98006    7.595363e+05
98007    6.103928e+05
98008    6.228193e+05
98010    3.681760e+05
98011    4.815349e+05
98014    3.634308e+05
98019    4.096115e+05
98022    2.710903e+05
98023    2.811750e+05
98024    4.065255e+05
98027    5.884460e+05
98028    4.616816e+05
98029    5.940526e+05
98030    2.944060e+05
98031    2.993259e+05
98032    2.512962e+05
98033    7.714699e+05
98034    5.069362e+05
98038    3.458791e+05
98039    1.814232e+06
98040    1.103000e+06
98042    3.011206e+05
98045    3.858291e+05
98052    6.418573e+05
98053    6.064355e+05
98055    3.032866e+05
98056    4.210656e+05
98058    3.412414e+05
98059    4.643719e+05
98065    5.074359e+05
98070    4.720914e+05
98072    5.225939e+05
98074    6.484587e+05
98075    7.664391e+05
98077    6.443466e+05
98092    3.103444e+05
98102    8.355160e+05
98103    5.831013e+05
98105    8.385226e+05
98106    3.186810e+05
98

## 4. Regressions

In [40]:
result1 = sm.ols("price ~ bedrooms + bathrooms + grade + sqft_living15", data = df2).fit()
result1.summary()

# bad adjusted R-squared

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.450
Model:                            OLS   Adj. R-squared:                  0.449
Method:                 Least Squares   F-statistic:                     4130.
Date:                Thu, 04 Aug 2022   Prob (F-statistic):               0.00
Time:                        18:32:34   Log-Likelihood:            -2.7868e+05
No. Observations:               20234   AIC:                         5.574e+05
Df Residuals:                   20229   BIC:                         5.574e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -7.784e+05   1.31e+04    -59.247      0.000   -8.04e+05   -7.53e+05
bedrooms       7668.8211   2223.883      3.448      0.001    3309.829     1.2e+04
bathrooms       1.76e+04   3251.886      5.412      0.000    1.12e+04     2.4e+04
grade           1.36e+05   2274.816     59.781      0.000    1.32e+05     1.4e+05
sqft_living15   104.4325      3.744     27.896      0.000      97.095     111.770
==============================================================================
Omnibus:                    11541.874   Durbin-Watson:                   1.954
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           190740.492
Skew:                           2.406   Prob(JB):                         0.00
Kurtosis:                      17.251   Cond. No.                     1.66e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.66e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [41]:
# same as result 1 but without y-intercept
result2 = sm.ols("price ~ bedrooms + bathrooms + grade + sqft_living15 -1", data = df2).fit()
result2.summary()

# good adjusted R-squared

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.827
Model:                            OLS   Adj. R-squared (uncentered):              0.827
Method:                 Least Squares   F-statistic:                          2.415e+04
Date:                Thu, 04 Aug 2022   Prob (F-statistic):                        0.00
Time:                        18:32:34   Log-Likelihood:                     -2.8030e+05
No. Observations:               20234   AIC:                                  5.606e+05
Df Residuals:                   20230   BIC:                                  5.606e+05
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
bedrooms      -3.889e+04   2253.660    -17.255      0.000   -4.33e+04   -3.45e+04
bathrooms      7.753e+04   3347.886     23.158      0.000     7.1e+04    8.41e+04
grade           2.25e+04   1329.000     16.927      0.000    1.99e+04    2.51e+04
sqft_living15   169.6254      3.876     43.761      0.000     162.028     177.223
==============================================================================
Omnibus:                    13292.825   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           257472.059
Skew:                           2.878   Prob(JB):                         0.00
Kurtosis:                      19.501   Cond. No.                     3.97e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 3.97e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [42]:
result3 = sm.ols("price ~ bedrooms + bathrooms + grade + sqft_living15 + view -1", data = df2).fit()
result3.summary()

# better adjusted R-squared

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.845
Model:                            OLS   Adj. R-squared (uncentered):              0.845
Method:                 Least Squares   F-statistic:                          2.199e+04
Date:                Thu, 04 Aug 2022   Prob (F-statistic):                        0.00
Time:                        18:32:34   Log-Likelihood:                     -2.7921e+05
No. Observations:               20234   AIC:                                  5.584e+05
Df Residuals:                   20229   BIC:                                  5.585e+05
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
bedrooms      -2.955e+04   2143.916    -13.785      0.000   -3.38e+04   -2.54e+04
bathrooms      7.218e+04   3173.720     22.744      0.000     6.6e+04    7.84e+04
grade          2.563e+04   1260.770     20.326      0.000    2.32e+04    2.81e+04
sqft_living15   133.8008      3.747     35.707      0.000     126.456     141.146
view           1.144e+05   2381.162     48.062      0.000     1.1e+05    1.19e+05
==============================================================================
Omnibus:                    12297.644   Durbin-Watson:                   1.951
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           218578.654
Skew:                           2.603   Prob(JB):                         0.00
Kurtosis:                      18.237   Cond. No.                     3.98e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 3.98e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [43]:
result3.predict({
    "bedrooms": 3,
    "bathrooms": 2,
    "grade": 8,
    "sqft_living15": 1800,
    "view": 0
})

#actual price was 510,000 - pretty good!

0    501549.279164
dtype: float64

In [44]:
result3.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 8,
    "sqft_living15": 1530,
    "view": 0
})

#actual price was 375,000 - bad prediction!

0    501513.909209
dtype: float64

In [45]:
# try model 3 again, using same zipcode

result3.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 9,
    "sqft_living15": 1630,
    "view": 0
})

#actual price was 880,000 - bad prediction!

0    540519.96489
dtype: float64

In [46]:
# try model 3 again, using more expensive zipcode 98103

result3.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 9,
    "sqft_living15": 1780,
    "view": 0
})

#actual price was 999,999 - bad prediction!

0    560590.091062
dtype: float64

In [47]:
# try model 3 again, using zipcode 98019

result3.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 8,
    "sqft_living15": 1790,
    "view": 0
})

#actual price was 330,000 - bad prediction!

0    536302.127906
dtype: float64

In [48]:
#same as model 3, but with sqft_living instead of sqft_living15

result6 = sm.ols("price ~ bedrooms + bathrooms + grade + sqft_living + view -1", data = df2).fit()
result6.summary()

# better adjusted R-squared than model 3

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.872
Model:                            OLS   Adj. R-squared (uncentered):              0.872
Method:                 Least Squares   F-statistic:                          2.751e+04
Date:                Thu, 04 Aug 2022   Prob (F-statistic):                        0.00
Time:                        18:32:34   Log-Likelihood:                     -2.7726e+05
No. Observations:               20234   AIC:                                  5.545e+05
Df Residuals:                   20229   BIC:                                  5.546e+05
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
bedrooms    -6.161e+04   2006.570    -30.703      0.000   -6.55e+04   -5.77e+04
bathrooms   -2.138e+04   3210.688     -6.659      0.000   -2.77e+04   -1.51e+04
grade        3.767e+04    987.422     38.148      0.000    3.57e+04    3.96e+04
sqft_living   233.0326      3.050     76.400      0.000     227.054     239.011
view         9.181e+04   2181.851     42.078      0.000    8.75e+04    9.61e+04
==============================================================================
Omnibus:                     9723.644   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           119283.645
Skew:                           2.002   Prob(JB):                         0.00
Kurtosis:                      14.201   Cond. No.                     4.62e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 4.62e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [49]:
result6.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 8,
    "sqft_living": 1530,
    "view": 0,
})

#actual price was 375,000 - not a bad prediction

0    419610.019841
dtype: float64

In [50]:
# try model 6 again, using same zipcode

result6.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 9,
    "sqft_living": 2870,
    "view": 0
})

#actual price was 880,000 - bad prediction!

0    769541.519579
dtype: float64

In [51]:
# try model 6 again, using more expensive zipcode 98103

result6.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 9,
    "sqft_living": 2100,
    "view": 0
})

#actual price was 999,999 - bad prediction!

0    590106.439483
dtype: float64

In [52]:
# try model 6 again, using zipcode 98019

result6.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 8,
    "sqft_living": 1600,
    "view": 0
})

#actual price was 330,000 - bad prediction!

0    435922.29985
dtype: float64

In [53]:
#same as model 6, but with sqft_living AND sqft_living15

result8 = sm.ols("price ~ bedrooms + bathrooms + grade + sqft_living + sqft_living15 + view -1", data = df2).fit()
result8.summary()

# same R-squared as model 6 - very interesting

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.872
Model:                            OLS   Adj. R-squared (uncentered):              0.872
Method:                 Least Squares   F-statistic:                          2.294e+04
Date:                Thu, 04 Aug 2022   Prob (F-statistic):                        0.00
Time:                        18:32:34   Log-Likelihood:                     -2.7726e+05
No. Observations:               20234   AIC:                                  5.545e+05
Df Residuals:                   20228   BIC:                                  5.546e+05
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
bedrooms      -6.145e+04   2006.583    -30.626      0.000   -6.54e+04   -5.75e+04
bathrooms     -2.095e+04   3212.465     -6.520      0.000   -2.72e+04   -1.46e+04
grade          3.565e+04   1155.002     30.868      0.000    3.34e+04    3.79e+04
sqft_living     227.4879      3.467     65.613      0.000     220.692     234.284
sqft_living15    13.0011      3.869      3.360      0.001       5.418      20.584
view           9.111e+04   2191.287     41.576      0.000    8.68e+04    9.54e+04
==============================================================================
Omnibus:                     9744.609   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           120003.348
Skew:                           2.006   Prob(JB):                         0.00
Kurtosis:                      14.235   Cond. No.                     6.29e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 6.29e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [54]:
result8.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 8,
    "sqft_living": 1530,
    "sqft_living15": 1530,
    "view": 0,
})

#actual price was 375,000 - not a bad prediction

0    416444.971415
dtype: float64

In [55]:
# try model 8 again, using same zipcode

result8.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 9,
    "sqft_living": 2870,
    "sqft_living15": 1630,
    "view": 0
})

#actual price was 880,000 - not terrible

0    758231.800241
dtype: float64

In [56]:
# try model 8 again, using more expensive zipcode 98103

result8.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 9,
    "sqft_living": 2100,
    "sqft_living15": 1780,
    "view": 0
})

#actual price was 999,999 - bad prediction!

0    585016.259724
dtype: float64

In [57]:
# try model 8 again, using zipcode 98019

result8.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 8,
    "sqft_living": 1600,
    "sqft_living15": 1790,
    "view": 0
})

#actual price was 330,000 - bad prediction!

0    435749.407633
dtype: float64

In [58]:
result4 = sm.ols("price ~ bedrooms + bathrooms + grade + sqft_living15 + view + sqft_lot15 -1", data = df2).fit()
result4.summary()

# no difference!

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.845
Model:                            OLS   Adj. R-squared (uncentered):              0.845
Method:                 Least Squares   F-statistic:                          1.841e+04
Date:                Thu, 04 Aug 2022   Prob (F-statistic):                        0.00
Time:                        18:32:35   Log-Likelihood:                     -2.7917e+05
No. Observations:               20234   AIC:                                  5.583e+05
Df Residuals:                   20228   BIC:                                  5.584e+05
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
bedrooms      -2.783e+04   2148.146    -12.953      0.000    -3.2e+04   -2.36e+04
bathrooms      6.735e+04   3212.047     20.968      0.000    6.11e+04    7.36e+04
grade          2.661e+04   1262.986     21.073      0.000    2.41e+04    2.91e+04
sqft_living15   144.0912      3.909     36.865      0.000     136.430     151.752
view           1.146e+05   2376.502     48.236      0.000     1.1e+05    1.19e+05
sqft_lot15       -2.7920      0.308     -9.053      0.000      -3.396      -2.187
==============================================================================
Omnibus:                    12404.225   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           226529.890
Skew:                           2.625   Prob(JB):                         0.00
Kurtosis:                      18.528   Cond. No.                     2.05e+04
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 2.05e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [59]:
result5 = sm.ols("price ~ bedrooms + bathrooms + grade + sqft_living15 + yr_last_reno -1", data = df2).fit()
result5.summary()

# better adjusted R-squared

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.857
Model:                            OLS   Adj. R-squared (uncentered):              0.857
Method:                 Least Squares   F-statistic:                          2.427e+04
Date:                Thu, 04 Aug 2022   Prob (F-statistic):                        0.00
Time:                        18:32:35   Log-Likelihood:                     -2.7836e+05
No. Observations:               20234   AIC:                                  5.567e+05
Df Residuals:                   20229   BIC:                                  5.568e+05
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
bedrooms       9362.9364   2176.272      4.302      0.000    5097.266    1.36e+04
bathrooms      2.212e+04   3157.362      7.005      0.000    1.59e+04    2.83e+04
grade          1.478e+05   2264.722     65.282      0.000    1.43e+05    1.52e+05
sqft_living15    97.8318      3.689     26.523      0.000      90.602     105.062
yr_last_reno   -441.8318      6.754    -65.423      0.000    -455.069    -428.594
==============================================================================
Omnibus:                    11435.071   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           192067.405
Skew:                           2.369   Prob(JB):                         0.00
Kurtosis:                      17.331   Cond. No.                     6.02e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 6.02e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [60]:
# checking to see predictions on a few random home prices

df2.tail()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,reno_living,sqft_lot_added_reno,increase_lot,decrease_lot,basement,yr_last_reno
21608,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509,3039,0,N,378,Y,N,N,2009
21609,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200,9030,-480,Y,1387,Y,N,N,2014
21610,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007,3027,0,N,657,Y,N,N,2009
21611,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287,2697,-190,Y,-1101,N,Y,N,2004
21612,20141015T000000,325000.0,2,0.75,1020,1076,2.0,0,0,3,7,1020,0,2008,0,98144,47.5941,-122.299,1020,1357,2377,0,N,281,Y,N,N,2008


In [61]:
result5.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 8,
    "sqft_living15": 1530,
    "view": 0,
    "yr_last_reno": 2009
})

#actual price was 375,000 - bad prediction!

0    528191.452798
dtype: float64

In [62]:
# try model 5 again, using same zipcode

result5.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 9,
    "sqft_living15": 1630,
    "view": 0,
    "yr_last_reno": 2014
})

#actual price was 880,000 - bad prediction!

0    683611.76799
dtype: float64

In [63]:
# try model 5 again, using more expensive zipcode 98103

result5.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 9,
    "sqft_living15": 1780,
    "view": 0,
    "yr_last_reno": 2008 
})

#actual price was 999,999 - bad prediction!

0    700937.529409
dtype: float64

In [64]:
# try model 5 again, using zipcode 98019

result5.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 8,
    "sqft_living15": 1790,
    "view": 0,
    "yr_last_reno": 2005
})

#actual price was 330,000 - bad prediction!

0    555395.049109
dtype: float64

In [65]:
result7 = sm.ols("price ~ bedrooms + bathrooms + grade + sqft_living15 + view + lat -1", data = df2).fit()
result7.summary()

# better adjusted R-squared

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.865
Model:                            OLS   Adj. R-squared (uncentered):              0.865
Method:                 Least Squares   F-statistic:                          2.160e+04
Date:                Thu, 04 Aug 2022   Prob (F-statistic):                        0.00
Time:                        18:32:35   Log-Likelihood:                     -2.7779e+05
No. Observations:               20234   AIC:                                  5.556e+05
Df Residuals:                   20228   BIC:                                  5.556e+05
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
bedrooms       1.104e+04   2128.825      5.184      0.000    6863.944    1.52e+04
bathrooms      1.879e+04   3111.697      6.038      0.000    1.27e+04    2.49e+04
grade          1.275e+05   2185.758     58.349      0.000    1.23e+05    1.32e+05
sqft_living15    79.3374      3.629     21.863      0.000      72.225      86.450
view            1.01e+05   2232.591     45.255      0.000    9.67e+04    1.05e+05
lat           -1.473e+04    266.430    -55.296      0.000   -1.53e+04   -1.42e+04
==============================================================================
Omnibus:                    10653.062   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           162640.916
Skew:                           2.181   Prob(JB):                         0.00
Kurtosis:                      16.187   Cond. No.                     4.46e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 4.46e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [66]:
# predicting same home prices as before to see the difference in accuracy between models 5 and 7
result7.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 8,
    "sqft_living15": 1530,
    "view": 0,
    "lat": 47.6993
})

#actual price was 375,000 - bad prediction!

0    519031.205316
dtype: float64

In [67]:
# try model 7 again, using same zipcode

result7.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 9,
    "sqft_living15": 1630,
    "view": 0,
    "lat": 47.6935
})

#actual price was 880,000 - bad prediction!

0    654586.115016
dtype: float64

In [68]:
# try model 7 again, using more expensive zipcode 98103

result7.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 9,
    "sqft_living15": 1780,
    "view": 0,
    "lat": 47.5983 
})

#actual price was 999,999 - bad prediction!

0    667889.254934
dtype: float64

In [69]:
# try model 7 again, using zipcode 98019

result7.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 8,
    "sqft_living15": 1790,
    "view": 0,
    "lat": 47.7736
})

#actual price was 330,000 - bad prediction!

0    538564.321538
dtype: float64

In [70]:
# includes pretty much all variables
result9 = sm.ols("price ~ bedrooms + bathrooms + sqft_living + sqft_lot + floors + view + condition + grade + sqft_above + sqft_basement + yr_built + lat + long + sqft_living15 + sqft_lot15 + sqft_total15 + sqft_living_added_reno + sqft_lot_added_reno + yr_last_reno -1", data = df2).fit()
result9.summary()

# good adjusted R-squared

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.911
Model:                            OLS   Adj. R-squared (uncentered):              0.911
Method:                 Least Squares   F-statistic:                          1.380e+04
Date:                Thu, 04 Aug 2022   Prob (F-statistic):                        0.00
Time:                        18:32:35   Log-Likelihood:                     -2.7357e+05
No. Observations:               20234   AIC:                                  5.472e+05
Df Residuals:                   20219   BIC:                                  5.473e+05
Df Model:                          15                                                  
Covariance Type:            nonrobust                                                  
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
bedrooms               -3.079e+04   1907.907    -16.137      0.000   -3.45e+04    -2.7e+04
bathrooms               3.041e+04   3175.931      9.575      0.000    2.42e+04    3.66e+04
sqft_living               70.3229      1.520     46.272      0.000      67.344      73.302
sqft_lot                  -8.2445      0.312    -26.445      0.000      -8.856      -7.633
floors                  2.101e+04   3436.414      6.113      0.000    1.43e+04    2.77e+04
view                    7.355e+04   1910.066     38.506      0.000    6.98e+04    7.73e+04
condition               2.885e+04   2153.764     13.396      0.000    2.46e+04    3.31e+04
grade                   1.082e+05   1992.499     54.291      0.000    1.04e+05    1.12e+05
sqft_above                39.8542      2.035     19.589      0.000      35.866      43.842
sqft_basement             30.4686      2.405     12.667      0.000      25.754      35.183
yr_built               -3465.9087    105.776    -32.766      0.000   -3673.239   -3258.579
lat                     4.977e+05   9312.101     53.450      0.000    4.79e+05    5.16e+05
long                    1.524e+05   3829.351     39.806      0.000    1.45e+05     1.6e+05
sqft_living15             38.4663      1.479     26.011      0.000      35.568      41.365
sqft_lot15               -16.5430      0.625    -26.471      0.000     -17.768     -15.318
sqft_total15              21.9233      0.867     25.276      0.000      20.223      23.623
sqft_living_added_reno   -31.8565      1.568    -20.322      0.000     -34.929     -28.784
sqft_lot_added_reno       -8.2985      0.318    -26.111      0.000      -8.921      -7.676
yr_last_reno             539.3821    110.954      4.861      0.000     321.903     756.861
==============================================================================
Omnibus:                    12879.279   Durbin-Watson:                   1.967
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           337039.716
Skew:                           2.636   Prob(JB):                         0.00
Kurtosis:                      22.287   Cond. No.                     1.07e+17
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenvalue is 1.32e-21. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

bedrooms + bathrooms + sqft_living + sqft_lot + floors + view + condition + grade + sqft_above + sqft_basement + yr_built + lat + long + sqft_living15 + sqft_lot15 + sqft_total15 + sqft_living_added_reno + sqft_lot_added_reno + yr_last_reno

In [71]:
# try model 9 again, using zipcode 98019

result9.predict({
    "bedrooms": 3,
    "bathrooms": 2.5,
    "grade": 8,
    "sqft_living15": 1790,
    "view": 0,
    "lat": 47.7736
})

#actual price was 330,000 - bad prediction!

PatsyError: predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Error evaluating factor: NameError: name 'sqft_living' is not defined
    price ~ bedrooms + bathrooms + sqft_living + sqft_lot + floors + view + condition + grade + sqft_above + sqft_basement + yr_built + lat + long + sqft_living15 + sqft_lot15 + sqft_total15 + sqft_living_added_reno + sqft_lot_added_reno + yr_last_reno -1
                                   ^^^^^^^^^^^

In [72]:
df2.loc[(df2["zipcode"] == 98004) & (df2["bedrooms"] ==3) & (df2["bathrooms"] == 2.5) & (df2["view"] == 0) & ((df2["sqft_living15"] >1500) & (df2["sqft_living15"]<2000))]

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,reno_living,sqft_lot_added_reno,increase_lot,decrease_lot,basement,yr_last_reno
5735,20150219T000000,990400.0,3,2.5,2100,4097,2.0,0,0,3,9,2100,0,2008,0,98004,47.5983,-122.2,1820,4764,6584,-280,Y,667,Y,N,N,2008
20991,20150226T000000,910000.0,3,2.5,2030,1160,3.0,0,0,3,9,1970,60,2007,0,98004,47.6213,-122.2,1950,1160,3110,-80,Y,0,N,N,Y,2007
21533,20150226T000000,999999.0,3,2.5,2100,4097,2.0,0,0,3,9,2100,0,2008,0,98004,47.5983,-122.2,1780,4764,6544,-320,Y,667,Y,N,N,2008


In [73]:
df2.loc[(df2["zipcode"] == 98103) & (df2["bedrooms"] ==3) & (df2["bathrooms"] == 2.5) & (df2["view"] == 0) & ((df2["sqft_living15"] >1500) & (df2["sqft_living15"]<2000))]

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,reno_living,sqft_lot_added_reno,increase_lot,decrease_lot,basement,yr_last_reno
562,20141217T000000,880000.0,3,2.5,2870,5163,2.0,0,0,3,9,2870,0,2014,0,98103,47.6935,-122.352,1630,7995,9625,-1240,Y,2832,Y,N,N,2014
3751,20140912T000000,416000.0,3,2.5,1710,1296,3.0,0,0,3,8,1510,200,2004,0,98103,47.6963,-122.342,1610,1282,2892,-100,Y,-14,N,Y,Y,2004
5794,20140716T000000,665000.0,3,2.5,1730,3000,2.0,0,0,3,8,1730,0,1996,0,98103,47.6532,-122.340,1730,1774,3504,0,N,-1226,N,Y,N,1996
10256,20140810T000000,720001.0,3,2.5,1430,2200,1.5,0,0,4,7,1430,0,1910,0,98103,47.6601,-122.331,1740,4275,6015,310,Y,2075,Y,N,N,1910
16322,20150407T000000,620000.0,3,2.5,1776,1248,3.0,0,0,3,8,1604,172,2006,0,98103,47.6539,-122.352,1780,1248,3028,4,Y,0,N,N,Y,2006
18551,20140508T000000,413450.0,3,2.5,1540,1614,3.0,0,0,3,8,1470,70,2008,0,98103,47.6961,-122.341,1540,1418,2958,0,N,-196,N,Y,Y,2008
19783,20141015T000000,414950.0,3,2.5,1570,1551,3.0,0,0,3,8,1570,0,2008,0,98103,47.6961,-122.341,1570,1705,3275,0,N,154,Y,N,N,2008
19830,20140502T000000,436110.0,3,2.5,1770,1235,3.0,0,0,3,8,1600,170,2007,0,98103,47.6965,-122.342,1680,1203,2883,-90,Y,-32,N,Y,Y,2007
20068,20140512T000000,635000.0,3,2.5,1570,1433,3.0,0,0,3,8,1570,0,2010,0,98103,47.6858,-122.336,1570,2652,4222,0,N,1219,Y,N,N,2010
20176,20150414T000000,660000.0,3,2.5,1780,1729,2.0,0,0,3,8,1080,700,2008,0,98103,47.6594,-122.355,1780,1741,3521,0,N,12,Y,N,Y,2008


In [74]:
df2.loc[(df2["zipcode"] == 98019) & (df2["bedrooms"] ==3) & (df2["bathrooms"] == 2.5) & (df2["view"] == 0) & ((df2["sqft_living15"] >1500) & (df2["sqft_living15"]<2000))]

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,reno_living,sqft_lot_added_reno,increase_lot,decrease_lot,basement,yr_last_reno
392,20140618T000000,330000.0,3,2.5,1600,26977,2.0,0,0,3,8,1600,0,2005,0,98019,47.7736,-121.901,1790,27743,29533,190,Y,766,Y,N,N,2005
4376,20150511T000000,350000.0,3,2.5,1640,10424,2.0,0,0,3,7,1640,0,1988,0,98019,47.7345,-121.977,1560,10101,11661,-80,Y,-323,N,Y,N,1988
6134,20140722T000000,350000.0,3,2.5,2100,3574,2.0,0,0,3,7,1690,410,2005,0,98019,47.7453,-121.984,1970,2962,4932,-130,Y,-612,N,Y,Y,2005
6137,20150223T000000,325000.0,3,2.5,1570,3143,2.0,0,0,3,7,1570,0,2001,0,98019,47.7364,-121.969,1740,3591,5331,170,Y,448,Y,N,N,2001
7915,20140822T000000,310000.0,3,2.5,1590,3359,2.0,0,0,3,7,1590,0,2000,0,98019,47.7349,-121.986,1820,3383,5203,230,Y,24,Y,N,N,2000
8112,20141118T000000,299950.0,3,2.5,1570,2577,2.0,0,0,3,7,1570,0,2005,0,98019,47.7456,-121.984,1970,2952,4922,400,Y,375,Y,N,N,2005
8226,20140814T000000,380600.0,3,2.5,1920,12244,2.0,0,0,3,7,1920,0,1998,0,98019,47.7256,-121.970,1920,11859,13779,0,N,-385,N,Y,N,1998
8463,20150127T000000,330000.0,3,2.5,2040,14071,2.0,0,0,3,7,2040,0,1995,0,98019,47.7278,-121.963,1890,14040,15930,-150,Y,-31,N,Y,N,1995
9804,20150403T000000,350000.0,3,2.5,1970,3655,2.0,0,0,3,7,1970,0,2003,0,98019,47.7453,-121.984,1970,2952,4922,0,N,-703,N,Y,N,2003
11192,20141119T000000,355000.0,3,2.5,1810,3192,1.0,0,0,3,7,1070,740,2001,0,98019,47.7364,-121.969,1740,3720,5460,-70,Y,528,Y,N,Y,2001


In [76]:
df2.loc[(df2["lat"]==47.6295)]

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,reno_living,sqft_lot_added_reno,increase_lot,decrease_lot,basement,yr_last_reno
8605,20140709T000000,594950.0,4,2.50,2720,10006,2.0,0,0,3,9,2720,0,1989,0,98074,47.6295,-122.042,2720,10759,13479,0,N,753,Y,N,N,1989
10769,20150407T000000,720000.0,3,2.50,2520,10012,2.0,0,0,3,10,2520,0,1987,0,98074,47.6295,-122.051,2680,10071,12751,160,Y,59,Y,N,N,1987
12151,20150411T000000,885000.0,4,3.75,2400,3520,1.0,0,0,3,7,1370,1030,1924,2005,98109,47.6295,-122.346,2230,1419,3649,-170,Y,-2101,N,Y,Y,2005
12826,20140725T000000,1749000.0,4,2.50,3910,22710,1.5,0,0,3,8,3910,0,1908,2003,98004,47.6295,-122.217,2920,16544,19464,-990,Y,-6166,N,Y,N,2003
13548,20150421T000000,1475000.0,3,2.50,2570,5000,2.0,0,0,3,11,2570,0,1984,0,98102,47.6295,-122.320,1570,5000,6570,-1000,Y,0,N,N,N,1984
14255,20150320T000000,1225000.0,6,2.25,2930,4320,2.0,0,0,3,9,2130,800,1913,0,98112,47.6295,-122.306,2860,4320,7180,-70,Y,0,N,N,Y,1913
16629,20140520T000000,598000.0,5,2.25,2890,12478,2.0,0,0,3,9,2890,0,1977,0,98074,47.6295,-122.052,2570,11880,14450,-320,Y,-598,N,Y,N,1977
18769,20140910T000000,585000.0,4,2.50,2370,15200,1.0,0,0,3,8,1660,710,1975,0,98052,47.6295,-122.089,2360,13879,16239,-10,Y,-1321,N,Y,Y,1975
20917,20140613T000000,575000.0,3,3.00,1384,1287,2.0,0,0,3,8,1144,240,2006,0,98102,47.6295,-122.320,1570,1288,2858,186,Y,1,Y,N,Y,2006


In [77]:
df2.to_csv("housepricesnooutliers.csv")

In [78]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20234 entries, 0 to 21612
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    20234 non-null  object 
 1   price                   20234 non-null  float64
 2   bedrooms                20234 non-null  int64  
 3   bathrooms               20234 non-null  float64
 4   sqft_living             20234 non-null  int64  
 5   sqft_lot                20234 non-null  int64  
 6   floors                  20234 non-null  float64
 7   waterfront              20234 non-null  int64  
 8   view                    20234 non-null  int64  
 9   condition               20234 non-null  int64  
 10  grade                   20234 non-null  int64  
 11  sqft_above              20234 non-null  int64  
 12  sqft_basement           20234 non-null  int64  
 13  yr_built                20234 non-null  int64  
 14  yr_renovated            20234 non-null

# 5. Run Regression on Training Set

(Since it's hard to see how good of a model the ones above were)

Used this site: https://stackabuse.com/linear-regression-in-python-with-scikit-learn/

### Iteration 1

In [87]:
df3 = df2.copy()

In [164]:
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [110]:
df3.drop(columns = "date", inplace = True)

In [115]:
df3.drop(columns = ["increase_lot", "decrease_lot", "basement"], inplace = True)

In [121]:
df3.drop(columns = "reno_living", inplace = True)

In [176]:
df3.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'sqft_total15', 'sqft_living_added_reno',
       'sqft_lot_added_reno', 'yr_last_reno'],
      dtype='object')

In [171]:
X = df3[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'sqft_total15', 'sqft_living_added_reno',
       'sqft_lot_added_reno', 'yr_last_reno']]

In [172]:
y = df3["price"]

In [173]:
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [174]:
regressor = linear_model.LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [177]:
feature_names = X.columns
model_coefficients = regressor.coef_

coefficients_df3 = pd.DataFrame(data = model_coefficients, 
                              index = feature_names, 
                              columns = ['Coefficient value'])
print(coefficients_df3)

                        Coefficient value
bedrooms                    -2.977512e+04
bathrooms                    2.609030e+04
sqft_living                 -5.245272e+12
sqft_lot                    -5.526547e+13
floors                       1.618615e+04
waterfront                   5.005839e+05
view                         5.183197e+04
condition                    2.957584e+04
grade                        1.017337e+05
sqft_above                  -2.822828e+12
sqft_basement               -2.822828e+12
yr_built                    -1.880950e+03
yr_renovated                 3.639750e+01
zipcode                     -5.146030e+02
lat                          5.940594e+05
long                        -1.509712e+05
sqft_living15               -1.364367e+13
sqft_lot15                   3.355370e+13
sqft_total15                 2.171177e+13
sqft_living_added_reno      -8.068099e+12
sqft_lot_added_reno         -5.526547e+13
yr_last_reno                -5.712913e+02


In [178]:
y_pred = regressor.predict(X_test)

In [179]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results)

          Actual     Predicted
2787    638500.0  5.491206e+05
16020  1280000.0  8.055311e+05
12463   795000.0  9.276845e+05
6553    498500.0  5.433428e+05
7519    330000.0  1.526839e+05
350     299000.0 -5.745161e+04
18348   255950.0  2.708543e+05
13725   766000.0  7.604873e+05
3797    586000.0  6.257550e+05
7643    460000.0  4.196267e+05
18596   645500.0  6.906783e+05
16126   550000.0  7.310284e+05
21372   490000.0  1.166054e+06
9804    350000.0  4.992779e+05
2910    510000.0  5.124364e+05
6340    480000.0  7.502678e+05
16460   197000.0  9.422005e+04
11823   419000.0  4.955963e+05
2416    286700.0  1.237360e+05
14523   750000.0  8.926724e+05
20553   344000.0  3.980186e+05
17828   623000.0  7.020995e+05
16575   461000.0  5.176708e+05
14565   380000.0  5.815150e+05
11677   218000.0  1.957766e+05
13164   425000.0  3.966374e+05
14567   400000.0  5.787865e+05
10768   603000.0  6.005071e+05
11043   751000.0  7.055639e+05
17365   826000.0  7.958865e+05
10715   415000.0  4.273550e+05
10029   

In [181]:
regressor.score(X_test, y_test)

0.6824893841794882

### Iteration 2

In [182]:
X = df3[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'sqft_total15', 'sqft_living_added_reno',
       'sqft_lot_added_reno', 'yr_last_reno']]

In [183]:
y = df3["price"]

In [184]:
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [185]:
regressor = linear_model.LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [186]:
feature_names = X.columns
model_coefficients = regressor.coef_

coefficients_df3 = pd.DataFrame(data = model_coefficients, 
                              index = feature_names, 
                              columns = ['Coefficient value'])
print(coefficients_df3)

                        Coefficient value
bedrooms                    -27850.488663
bathrooms                    26948.799386
sqft_living                     71.309687
sqft_lot                        -8.766976
floors                       13199.732360
waterfront                  549116.549254
view                         50263.589381
condition                    29368.528443
grade                       102111.503958
sqft_above                      47.386070
sqft_basement                   23.923617
yr_built                     -1677.277557
yr_renovated                    48.469328
zipcode                       -499.509959
lat                         588060.678154
long                       -148565.357606
sqft_living15                   41.280947
sqft_lot15                     -17.653001
sqft_total15                    23.627945
sqft_living_added_reno         -30.028740
sqft_lot_added_reno             -8.886021
yr_last_reno                  -790.428814


In [187]:
y_pred = regressor.predict(X_test)

In [188]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results)

          Actual     Predicted
19267   495000.0  3.724822e+05
19035   512000.0  6.462229e+05
9818    753000.0  7.407040e+05
12105   502000.0  5.476023e+05
17171   250000.0  3.391327e+05
11432   745000.0  5.763133e+05
17051   190000.0  2.015497e+05
7626    730000.0  6.280961e+05
14752   605000.0  6.619924e+05
2759    172000.0  2.418768e+05
1826    450000.0  3.266437e+05
7891    406550.0  4.849606e+05
13165   548000.0  6.279919e+05
12890   400000.0  4.691828e+05
5590    725000.0  4.512566e+05
2383    650000.0  1.194154e+06
4625    419000.0  3.752148e+05
9753    246000.0  3.337200e+05
18773   322500.0  3.725700e+05
4355    525000.0  4.608170e+05
17188   502000.0  5.368328e+05
19780   538000.0  6.236175e+05
2529    392450.0  6.514629e+05
10807   729000.0  7.361844e+05
15156   169000.0  3.939580e+05
19188   345600.0  9.674634e+05
10140   112000.0  1.136110e+05
1980    635000.0  9.035685e+05
9343    582500.0  3.545206e+05
14567   400000.0  5.758415e+05
7491    365000.0  4.257203e+05
6116    

In [189]:
regressor.score(X_test, y_test)

0.6987101836503827

### Iteration 3

In [196]:
df2.loc[(df2["zipcode"]==98039) & (df2["price"] <1000000) & (df2["bedrooms"]>3)]

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_total15,sqft_living_added_reno,reno_living,sqft_lot_added_reno,increase_lot,decrease_lot,basement,yr_last_reno
12811,20140804T000000,787500.0,4,1.75,1580,9382,1.0,0,0,3,7,1080,500,1963,0,98039,47.6353,-122.232,2010,9382,11392,430,Y,0,N,N,Y,1963
